In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from stack_segmentation.stack import Stack

In [5]:
from stack_segmentation.aug_pipelines import medium_aug
from stack_segmentation.io import make_dataloader, collate_fn_basic
from stack_segmentation.training import handle_stacks_data, make_model, train_loop
from stack_segmentation.unet import UNet
from stack_segmentation.pipeline_config import dataloaders_conf, model_conf, train_conf, loss_config

In [6]:
from exp_config import data_conf

## Parameters to tune

In [7]:
train_conf['device'] = 'cuda:1'
model_conf['device'] = 'cuda:1'

In [10]:
data_conf['conf_name'] = 'basic_lr1e-2_epoch300_soft_aug_k_1'
data_conf

{'conf_name': 'basic_lr1e-2_epoch300_soft_aug_k_1',
 'stacks': [{'path': '../../data/carb96558',
   'slice_train': (slice(None, None, None),
    slice(None, None, None),
    slice(None, 230, None)),
   'slice_val': (slice(None, None, None),
    slice(None, None, None),
    slice(250, 470, None))},
  {'path': '../../data/SoilB-2',
   'slice_train': (slice(None, None, None),
    slice(None, None, None),
    slice(None, 230, None)),
   'slice_val': (slice(None, None, None),
    slice(None, None, None),
    slice(240, 460, None))},
  {'path': '../../data/Urna_22',
   'slice_train': (slice(None, None, None),
    slice(None, None, None),
    slice(None, 220, None)),
   'slice_val': (slice(None, None, None),
    slice(None, None, None),
    slice(245, 455, None))},
  {'path': '../../data/carb96558',
   'slice_test': (slice(None, None, None),
    slice(None, None, None),
    slice(490, None, None))},
  {'path': '../../data/carb71',
   'slice_test': (slice(None, None, None),
    slice(None, Non

In [11]:
from stack_segmentation.aug_pipelines import soft_aug
dataloaders_conf['train']['augmentation_pipeline'] = soft_aug(original_height=128, original_width=128, k=1)
dataloaders_conf

{'train': {'batch_size': 32,
  'num_workers': 8,
  'shuffle': True,
  'augmentation_pipeline': Compose([
    OneOf([
      RandomSizedCrop(always_apply=False, p=0.5, min_max_height=(128, 128), height=128, width=128, w2h_ratio=1.0, interpolation=1),
      PadIfNeeded(always_apply=False, p=0.5, min_height=128, min_width=128, border_mode=4, value=None, mask_value=None),
    ], p=1),
    VerticalFlip(always_apply=False, p=0.5),
    HorizontalFlip(always_apply=False, p=0.5),
    RandomRotate90(always_apply=False, p=0.5),
    Transpose(always_apply=False, p=0.5),
  ], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})},
 'val': {'batch_size': 32,
  'num_workers': 8,
  'shuffle': False,
  'augmentation_pipeline': None},
 'test': {'batch_size': 32,
  'num_workers': 8,
  'shuffle': True,
  'augmentation_pipeline': None}}

In [12]:
train_conf['num_epochs'] = 300
train_conf

{'num_epochs': 300, 'device': 'cuda:1'}

In [13]:
model_conf['opt_type'] = 'SGD'
model_conf['lr'] = 1e-2
model_conf['weight'] = None
model_conf['nesterov'] = True
model_conf

{'device': 'cuda:1',
 'opt_type': 'SGD',
 'lr': 0.01,
 'weight_decay': 0.0001,
 'amsgrad': False,
 'nesterov': True,
 'momentum': 0.9,
 'centered': False,
 'min_lr': 1e-06,
 'factor': 0.5,
 'patience': 5,
 'weight': None}

In [14]:
loss_config =[
    {'loss': 'BCE', 'weight': 1, 'params': {}},
]
loss_config

[{'loss': 'BCE', 'weight': 1, 'params': {}}]

In [15]:
data_conf['patches']

{'train': (128, 128, 1), 'val': (128, 128, 1), 'test': (128, 128, 1)}

## Prepare train, validation and test data

In [16]:
data_train, data_val, data_test = handle_stacks_data(**data_conf)

720it [00:03, 212.64it/s]
100%|██████████| 720/720 [00:07<00:00, 94.77it/s] 
8280it [00:00, 101687.84it/s]
7920it [00:00, 240517.89it/s]
700it [00:03, 222.66it/s]
100%|██████████| 700/700 [00:06<00:00, 102.16it/s]
8280it [00:00, 219431.95it/s]
7920it [00:00, 241943.83it/s]
710it [00:03, 216.46it/s]
100%|██████████| 710/710 [00:07<00:00, 99.08it/s] 
7920it [00:00, 250029.26it/s]
7560it [00:00, 81040.45it/s]
720it [00:00, 800.20it/s]
100%|██████████| 720/720 [00:07<00:00, 96.93it/s] 
8280it [00:00, 244626.13it/s]
720it [00:03, 228.52it/s]
100%|██████████| 720/720 [00:07<00:00, 98.12it/s] 
25920it [00:00, 138492.88it/s]
700it [00:02, 248.33it/s]
100%|██████████| 700/700 [00:06<00:00, 103.21it/s]
25200it [00:00, 228473.61it/s]
509it [00:01, 384.57it/s]
100%|██████████| 509/509 [00:02<00:00, 196.48it/s]
8144it [00:00, 65628.67it/s]
700it [00:03, 231.79it/s]
100%|██████████| 700/700 [00:06<00:00, 102.99it/s]
25200it [00:00, 232642.49it/s]
700it [00:00, 833.57it/s]
100%|██████████| 700/700 [0

In [17]:
len(data_train), len(data_val), len(data_test)

(24480, 23400, 11)

In [18]:
dataloader_train = make_dataloader(
    samples=data_train, 
    collate_fn=collate_fn_basic,
    **dataloaders_conf['train']
)

dataloader_val = make_dataloader(
    samples=data_val, 
    collate_fn=collate_fn_basic,
    **dataloaders_conf['val']
)

dataloaders_test = {
    name: make_dataloader(
        samples=data, 
        collate_fn=collate_fn_basic,
        **dataloaders_conf['test']
    ) for name, data in data_test.items()}

## Create model and metrics

In [19]:
from stack_segmentation.metrics import accuracy, precision, recall, f1, pr_auc, iou

In [21]:
device = 'cuda:1'

In [22]:
model, criterion, optimizer, scheduler = make_model(loss_config=loss_config, **model_conf)

## Run experiment

In [20]:
metrics = {
    'accuracy': accuracy, 
    'precision': precision, 
    'recall': recall, 
    'f1': f1,
    'pr_auc': pr_auc, 
    'iou': iou,
}

In [ ]:
results = train_loop(
    model=model,
    dataloader_train=dataloader_train, 
    dataloader_val=dataloader_val,
    dataloaders_test=dataloaders_test,
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    metrics=metrics,
    exp_name=data_conf['conf_name'],
    **train_conf)

  0%|          | 0/765 [00:00<?, ?it/s]

Epoch 0...


  0%|          | 0/732 [00:00<?, ?it/s]

Mean train loss: 0.046717


  0%|          | 0/765 [00:00<?, ?it/s]

...

  0%|          | 0/732 [00:00<?, ?it/s]

Mean train loss: 0.011634


  0%|          | 0/765 [00:00<?, ?it/s]

Mean val loss: 0.012284
EarlyStopping counter: 6 out of 10
Epoch 53...


 54%|█████▍    | 416/765 [01:24<01:09,  5.00it/s]

## Dump experiment results

In [ ]:
import pickle
import json

In [ ]:
p = './{}_exp_results.pkl'.format(data_conf['conf_name'])
with open(p, 'wb') as f:
    pickle.dump(results, f)

In [24]:
# p = './{}_exp_results.pkl'.format(data_conf['conf_name'])
# with open(p, 'rb') as f:
#     results = pickle.load(f)

In [11]:
import torch
model.load_state_dict(torch.load('./{}.pt'.format(data_conf['conf_name'])))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Train and validation losses

In [ ]:
from itertools import chain

In [ ]:
train_losses = list(chain(*[item for item in results['train_losses']]))
val_losses = list(chain(*[item for item in results['val_losses']]))

In [ ]:
def moving_average(a, n=5) :
    ret = np.cumsum([a[0]] * (n - 1) + a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
plt.figure(figsize=(10, 10))
plt.title('Moving-averaged batch losses')
plt.plot(np.arange(len(train_losses)), moving_average(train_losses), label='train')
plt.plot(np.arange(len(val_losses)), moving_average(val_losses), label='validation')

plt.legend(loc='best')
plt.yscale('log')

# plt.ylim([1e-2, 1])
plt.show()

In [ ]:
mean_train_loss = [np.mean(item) for item in results['train_losses']]
mean_val_loss = [np.mean(item) for item in results['val_losses']]

In [ ]:
plt.figure(figsize=(10, 10))
plt.title('Epoch losses')
plt.plot(np.arange(len(mean_train_loss)) + 1, mean_train_loss, label='train')
plt.plot(np.arange(len(mean_val_loss)) + 1, mean_val_loss, label='val')

plt.yscale('log')
plt.legend(loc='best')

plt.xlim([1, len(mean_train_loss) + 1])
plt.show()

## Results

In [ ]:
import pandas as pd

In [ ]:
from visualization_utils import make_df

In [ ]:
df = make_df(results, model_name='basic')
df

In [ ]:
print('Mean   IOU: {:.5}'.format(df['iou'].mean()))
print('Std    IOU: {:.5}'.format(df['iou'].std()))
print('Min    IOU: {:.5}'.format(df['iou'].min()))
print('Median IOU: {:.5}'.format(df['iou'].median()))